In [ ]:
import pandas as pd
import numpy as np
import scipy.interpolate
from math import log, exp
import datetime
from pandas.tseries.offsets import BDay

In [ ]:
df = pd.read_csv("/content/dataset.csv")
df

,Maturity,Germany,Italy
0,1Y,2.140,2.532
1,2Y,2.109,2.665
2,3Y,2.046,2.915
3,4Y,1.985,3.053
4,5Y,1.995,3.361
5,6Y,1.955,3.360
6,7Y,2.009,3.459
7,8Y,1.970,3.656
8,9Y,1.967,3.771
9,10Y,2.020,3.895


In [ ]:
years = []
for index, row in df.iterrows():
  year = int(row['Maturity'].replace("Y", ''))
  years.append(year)
df.insert(loc = 1, column = "Years", value = years)

In [ ]:
rec_rate = 0.4
defaults = []
for index, row in df.iterrows():
  default = (1 - pow(1 + row["Germany"] / 100, row["Years"]) / pow(1 + row["Italy"] / 100, row["Years"])) / (1 - rec_rate)
  defaults.append(np.round((100 * default), decimals = 2))
df.insert(loc = 4, column = "Default Probability", value = defaults)

In [ ]:
f = scipy.interpolate.interp1d(df["Years"], df["Default Probability"], 'linear')
g = scipy.interpolate.interp1d(df["Years"], df["Germany"], 'linear')
i = scipy.interpolate.interp1d(df["Years"], df["Italy"], 'linear')

for row in range(11, 15):
  df.loc[row - 0.5] = [str(row) + 'Y', row, np.round(g(row), decimals = 3), np.round(i(row), decimals = 3), np.round(f(row), decimals = 2)]
for row in range(16, 20):
  df.loc[row - 0.5] = [str(row) + 'Y', row, np.round(g(row), decimals = 3), np.round(i(row), decimals = 3), np.round(f(row), decimals = 2)]
for row in range(21, 25):
  df.loc[row - 0.5] = [str(row) + 'Y', row, np.round(g(row), decimals = 3), np.round(i(row), decimals = 3), np.round(f(row), decimals = 2)]
for row in range(26, 30):
  df.loc[row - 0.5] = [str(row) + 'Y', row, np.round(g(row), decimals = 3), np.round(i(row), decimals = 3), np.round(f(row), decimals = 2)]

In [ ]:
df = df.sort_values(by = "Years")
df["Survival probability"] = (100 - df["Default Probability"])
column_to_move = df.pop("Survival probability")
df.insert(4, "Survival probability", column_to_move)

In [ ]:
intervals = []
hazards = []
intervals.append(np.round(100 - df.iloc[0,4], decimals = 2))
hazards.append(np.round((-log(df.iloc[0,4] / 100) / df.iloc[0,1]) * 100, decimals = 2))
for i in range(1, len(df)):
  intervals.append(np.round(df.iloc[i - 1,4] - df.iloc[i,4], decimals = 2))
  hazards.append(np.round(log(df.iloc[i - 1,4] / df.iloc[i,4]) / (df.iloc[i,1] - df.iloc[i - 1, 1]) * 100, decimals = 2))
df["Interval PD"] = intervals
df["Hazard Rate"] = hazards

In [ ]:
start = datetime.date(2022, 11, 22)
end = datetime.date(2027, 12, 20)
dates = pd.bdate_range(start, end)
df_swap = pd.DataFrame()
df_swap["Date"] = dates
prev_coupon = []
bdays = BDay()
def is_business_day(date):
    return date + 0*bdays
prev_coupon.append(datetime.date(2022, 11, 22))
for j in range(2022, 2027):
  date = datetime.date(j, 12, 20)
  prev_coupon.append(is_business_day(date))
  for i in range(3, 12, 3):
    date = datetime.date(j + 1, i, 20)
    prev_coupon.append(is_business_day(date))
  j += 1
prev_coupon.append(datetime.date(2027, 12, 20))
prev_coupon = pd.to_datetime(prev_coupon).date

In [ ]:
df_swap['Date'] = pd.to_datetime(df_swap['Date']).dt.date
year_prob = []
year_prob.append(0)
surv_prob = []
surv_prob.append(100)
interval_PD = []
interval_PD.append(0)
m = 0
for i in range(1, len(df_swap)):
  delta = df_swap.iloc[i, 0] - start
  year_prob.append(np.round(delta.days / 365.25, decimals = 3))
  day = df_swap.iloc[i, 0].day
  if df_swap.iloc[i, 0].month == start.month:
    if day == start.day:
      m += 1
    elif day == 21:
      m += 1
    elif day == 22:
      m += 1
  surv_prob.append(np.round(surv_prob[i - 1] * exp((-df.iloc[m, 7] / 100) * (year_prob[i] - year_prob[i - 1])), decimals = 4))
  interval_PD.append(np.round(surv_prob[i - 1] - surv_prob[i], decimals = 4))

In [ ]:
df_swap["Years"] = year_prob
df_swap["Survival Probability"] = surv_prob
df_swap["Interval PD"] = interval_PD

In [ ]:
cds = 0.025
notional = 1000000
premium = 600000
const = cds * notional
accrueds = []
accrueds.append(0)
exp_pv = []
exp_pv.append(0)
exp_pv_s = []
exp_pv_s.append(0)
j = 0
p = 0
m = 0
for i in range(1, len(df_swap)):
    if df_swap.iloc[i, 0] == prev_coupon[j + 1] and j != len(prev_coupon):
      delta = df_swap.iloc[i, 0] - prev_coupon[j]
      accrued = np.round((const * delta.days) / 365.25, decimals = 2)
      accrueds.append(accrued)
      exp_pv.append(accrued * (df_swap.iloc[i, 2] + df_swap.iloc[i, 3]) / 100)
      j += 1
    else:
      delta = df_swap.iloc[i, 0] - prev_coupon[j]
      accrued = np.round((const * delta.days) / 365.25, decimals = 2)
      accrueds.append(accrued)
      exp_pv.append(np.round(accrued * df_swap.iloc[i, 3] / 100, decimals = 3))
    exp_pv_s.append(np.round(premium * df_swap.iloc[i, 3] / 100, decimals = 3))

In [ ]:
df_swap["Accrued"] = accrueds
df_swap["Buyer expected PV"] = exp_pv
df_swap["Seller expected PV"] = exp_pv_s

In [ ]:
df_swap

,Date,Years,Survival Probability,Interval PD,Accrued,Buyer expected PV,Seller expected PV
0,2022-11-22,0.000,100.0000,0.0000,0.00,0.000000,0.0
1,2022-11-23,0.003,99.9981,0.0019,68.45,0.001000,11.4
2,2022-11-24,0.005,99.9968,0.0013,136.89,0.002000,7.8
3,2022-11-25,0.008,99.9949,0.0019,205.34,0.004000,11.4
4,2022-11-28,0.016,99.9898,0.0051,410.68,0.021000,30.6
...,...,...,...,...,...,...,...
1320,2027-12-14,5.060,87.7472,0.0078,5817.93,0.454000,46.8
1321,2027-12-15,5.062,87.7420,0.0052,5886.38,0.306000,31.2
1322,2027-12-16,5.065,87.7342,0.0078,5954.83,0.464000,46.8
1323,2027-12-17,5.068,87.7264,0.0078,6023.27,0.470000,46.8


In [ ]:
df.to_excel("/content/Default Probability.xlsx")
df_swap.to_excel("/content/Credit Default Swap.xlsx")